## Feature engineering part

In [ ]:
import pandas as pd
import requests
import yfinance as yf
from bs4 import BeautifulSoup

In [2]:
url = "https://stockanalysis.com/list/nasdaq-100-stocks/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")

table = soup.find('table')
rows = table.find_all('tr')

# Make sure to call find_all on the first row only for headers
headers = [th.text.strip() for th in rows[0].find_all('th')]
data = []
for row in rows[1:]:
    cols = [td.text.strip() for td in row.find_all('td')]
    if cols:
        data.append(cols)

df = pd.DataFrame(data, columns=headers)
print(df.head())


  No. Symbol           Company Name Market Cap Stock Price % Change  Revenue
0   1   NVDA     NVIDIA Corporation  4,257.60B      175.21   -1.43%  165.22B
1   2   MSFT  Microsoft Corporation  3,817.45B      513.57   -0.35%  281.72B
2   3   AAPL             Apple Inc.  3,547.08B      239.02    0.98%  408.63B
3   4   GOOG          Alphabet Inc.  3,043.79B      251.25   -0.20%  371.40B
4   5  GOOGL          Alphabet Inc.  3,035.59B      251.00   -0.24%  371.40B


In [3]:
tickers_list =  df['Symbol'].tolist()[0:1]
start_date = '2023-01-01'
end_date = '2024-01-01'

In [4]:
for i in tickers_list:
  data = yf.Ticker(i).history(start=start_date, end=end_date, interval='1d')
  print(data.shape)

(250, 7)
